In [1]:
import torch
import numpy as np
import sys
sys.path.append('../../')

In [2]:
from model.correlator import ContextualRegressor
from model.dataset import GaussianSimulator, Dataset
from model.trainer import Trainer, MSE

## Few distributions, many samples

In [11]:
p, k, c = 4, 4, 10
n = 100
simulator = GaussianSimulator(p, k, c)
C, X = simulator.gen_samples(n)
db = Dataset(C, X)
X.shape

(400, 4)

In [12]:
model = ContextualRegressor(db.C.shape, db.T.shape, num_archetypes=100, final_dense_size=20)
opt = torch.optim.Adam
trainer = Trainer(model, opt, db)

In [13]:
print(trainer.test())
trainer.train(100)
print(trainer.test())

1.486844539642334
0.759985089302063


In [14]:
C, T, X = trainer.db.get_test()
beta, mu = trainer.model(C, T)
x = X[:,0]
y = X[:,1]
fx = beta.squeeze() * x + mu.squeeze()
x = x.detach().numpy()
y = y.detach().numpy()
fx = fx.detach().numpy()
np.var(fx)/np.var(y), np.var(y - fx)/np.var(y)

(0.36571705, 0.6281906)

In [25]:
from sklearn.linear_model import LinearRegression
X_train = trainer.db.X[trainer.db.train_idx].detach().numpy()
xtrain = X_train[:,0]
ytrain = X_train[:,1]
reg = LinearRegression().fit(xtrain.reshape(-1, 1), ytrain)
fx_global = reg.predict(x.reshape(-1, 1))
np.var(fx_global)/np.var(y), np.var(y - fx_global)/np.var(y)

(0.109446846, 0.9067758)

## Many distributions, single-samples

In [29]:
p, k, c = 4, 400, 10
n = 1
simulator = GaussianSimulator(p, k, c)
C, X = simulator.gen_samples(n)
db = Dataset(C, X)
X.shape

(400, 4)

In [30]:
model = ContextualRegressor(db.C.shape, db.T.shape, num_archetypes=100, final_dense_size=20)
opt = torch.optim.Adam
trainer = Trainer(model, opt, db)

In [31]:
print(trainer.test())
trainer.train(100)
print(trainer.test())

3.104039192199707
0.9570620656013489


In [32]:
C, T, X = trainer.db.get_test()
beta, mu = trainer.model(C, T)
x = X[:,0]
y = X[:,1]
fx = beta.squeeze() * x + mu.squeeze()
x = x.detach().numpy()
y = y.detach().numpy()
fx = fx.detach().numpy()
np.var(fx)/np.var(y), np.var(y - fx)/np.var(y)

(0.34410098, 0.7189247)

In [33]:
from sklearn.linear_model import LinearRegression
X_train = trainer.db.X[trainer.db.train_idx].detach().numpy()
xtrain = X_train[:,0]
ytrain = X_train[:,1]
reg = LinearRegression().fit(xtrain.reshape(-1, 1), ytrain)
fx_global = reg.predict(x.reshape(-1, 1))
np.var(fx_global)/np.var(y), np.var(y - fx_global)/np.var(y)

(0.051768966, 0.95044315)